In [1]:
import gzip
import json
import re

# —————————————————————————
# 1. 「イギリス」記事の本文を取得
# —————————————————————————
path = 'jawiki-country.json.gz'
with gzip.open(path, 'rt', encoding='utf-8') as f:
    text = None
    for line in f:
        dic = json.loads(line)
        if dic['title'] == 'イギリス':
            text = dic['text']
            break
if text is None:
    raise RuntimeError('「イギリス」記事が見つかりませんでした。')

# —————————————————————————
# 2. 基礎情報テンプレートを抜き出し
# —————————————————————————
tpl_re = re.compile(r'^\{\{基礎情報.+?$(.*?)^\}\}$',
                    re.MULTILINE | re.DOTALL)
m = tpl_re.search(text)
if not m:
    raise RuntimeError('基礎情報テンプレートが見つかりませんでした。')
tpl_body = m.group(1)

# —————————————————————————
# 3. フィールドを辞書へパース
# —————————————————————————
fields = dict(re.findall(r'^\|\s*(.+?)\s*=\s*(.+)$',
                         tpl_body, flags=re.MULTILINE))

# —————————————————————————
# 4. マークアップ除去・整形
# —————————————————————————
clean = {}
for k, v in fields.items():
    s = v

    # ① 強調マークアップ（'' ''', '''''）を除去
    s = re.sub(r"'{2,5}", '', s)

    # ② 内部リンク [[A|B]] または [[A]] を「B」/「A」に
    s = re.sub(r'\[\[(?:[^|\]]*?\|)?([^\]]+?)\]\]', r'\1', s)

    # ③ ファイルリンク [[File:…|…|キャプション]] → キャプション
    s = re.sub(
        r'\[\[(?:File|ファイル):(?:.*?\|)*(.+?)(?:\|.*)?\]\]',
        r'\1',
        s,
        flags=re.IGNORECASE
    )

    # ④ 外部リンク [http(s)://… 表示文字] → 表示文字
    s = re.sub(r'\[https?://[^\s\]]+\s+([^\]]+?)\]', r'\1', s)
    #    外部リンク [http(s)://…] → URL のまま
    s = re.sub(r'\[(https?://[^\]]+)\]', r'\1', s)

    # ⑤ HTML コメントを削除
    s = re.sub(r'<!--.*?-->', '', s, flags=re.DOTALL)

    # ⑥ <ref>…</ref> や <ref …/> タグを削除
    s = re.sub(r'<ref[^>]*?>.*?</ref>', '', s, flags=re.DOTALL)
    s = re.sub(r'<ref[^/]*/>', '', s)

    # ⑦ テンプレート {{…}} を大まかに除去（多段ネストには弱い）
    s = re.sub(r'\{\{(?:[^{}]|\{\{[^}]*\}\})*?\}\}', '', s)

    # ⑧ HTMLタグ <…> を削除
    s = re.sub(r'<[^>]+>', '', s)

    # ⑨ URL 生文字列（必要なら削除／そのまま残す）
    #    s = re.sub(r'https?://\S+', '', s)

    clean[k] = s.strip()

# —————————————————————————
# 5. 整形後の基礎情報を表示
# —————————————————————————
for key, val in clean.items():
    print(f'{key}: {val}')


略名: イギリス
日本語国名: グレートブリテン及び北アイルランド連合王国
公式国名: 
国旗画像: Flag of the United Kingdom.svg
国章画像: 85px|イギリスの国章
国章リンク: （国章）
標語: （フランス語:神と我が権利）
国歌: 神よ女王を護り賜え
地図画像: Europe-UK.svg
位置画像: United Kingdom (+overseas territories) in the World (+Antarctica claims).svg
公用語: 英語
首都: ロンドン（事実上）
最大都市: ロンドン
元首等肩書: 女王
元首等氏名: エリザベス2世
首相等肩書: 首相
首相等氏名: ボリス・ジョンソン
他元首等肩書1: 貴族院議長
他元首等氏名1: ノーマン・ファウラー
他元首等肩書2: 庶民院議長
他元首等氏名2: 
他元首等肩書3: 最高裁判所長官
他元首等氏名3: ブレンダ・ヘイル
面積順位: 76
面積大きさ: 1 E11
面積値: 244,820
水面積率: 1.3%
人口統計年: 2018
人口順位: 22
人口大きさ: 1 E7
人口値: 6643万5600
人口密度値: 271
GDP統計年元: 2012
GDP値元: 1兆5478億
GDP統計年MER: 2012
GDP順位MER: 6
GDP値MER: 2兆4337億
GDP統計年: 2012
GDP順位: 6
GDP値: 2兆3162億
GDP/人: 36,727
建国形態: 建国
確立形態1: イングランド王国／スコットランド王国（両国とも1707年合同法まで）
確立年月日1: 927年／843年
確立形態2: グレートブリテン王国成立（1707年合同法）
確立年月日2: 1707年5月1日
確立形態3: グレートブリテン及びアイルランド連合王国成立（1800年合同法）
確立年月日3: 1801年1月1日
確立形態4: 現在の国号「グレートブリテン及び北アイルランド連合王国」に変更
確立年月日4: 1927年4月12日
通貨: UKポンド (£)
通貨コード: GBP
時間帯: ±0
夏時間: +1
ISO 3166-1: GB / GBR
ccTLD: .uk / .gb
国際電話番号: 44
注記: 
